In [1]:
!pip install transformers
!pip install catboost
!pip install sacremoses

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 31.4 MB/s 
     |████████████████████████████████| 163 kB 70.2 MB/s 
     |████████████████████████████████| 7.6 MB 57.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.8 MB 1.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 880 kB 37.2 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=05ee81a7abf3ee4a3a6707f8fe1c46aba311e1bb17451d19da47859c1765cbd7
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses


In [2]:
import zipfile
with zipfile.ZipFile('/content/docQA_14_10.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/docQA_14_10')

In [2]:
cd /content/docQA_14_10

/content/docQA_14_10


In [ ]:

# .doc via antiword
# .pdf via pdftotext (default) or pdfminer.six
# .docx via python-docx
# .rtf via unrtf


Генерация вопросов


In [2]:
from docQA.pipelines import Pipeline, TranslatorPipeline, RetrieverPipeline, RankerPipeline, CatboostPipeline, QgPipeline
import pandas as pd

pipe = Pipeline(['docs/Федеральный-закон-от-27.07.2006-N-152-ФЗ-О-персональных-данных.txt'])
pipe.add_node(TranslatorPipeline, name='translator', is_technical=True, model_name='facebook/wmt19-ru-en',translate_text=False)
pipe.add_node(RetrieverPipeline, name='retriever')
pipe.add_node(RankerPipeline, name='ranker')

qgpipe = Pipeline(['docs/Федеральный-закон-от-27.07.2006-N-152-ФЗ-О-персональных-данных.txt'])
qgpipe.add_node(QgPipeline, cdqa_pipe=pipe, name='qg')

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

Opening docs:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


...

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/634 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

In [3]:
qgpipe(data='/content/generated_quesions_3.csv', return_output=False)

Generating questions:   0%|          | 0/342 [00:00<?, ?it/s]

Основная модель


In [4]:
df = pd.read_csv('/content/generated_quesions_3.csv')
newDf = []
for i in range(len(df)):
    if '<unk>' not in df.iloc[i]['en_question']:
        newDf.append(df.iloc[i])
newDf = pd.DataFrame(newDf)

In [5]:
newDf

,Unnamed: 0,en_question,en_context,native_context,retriever_score,ranker_score,rank,en_question_answer
1,1,What is the name of the person who requested t...,3. The information specified in paragraph 7 of...,"3. Сведения, указанные в части 7 настоящей ста...",0.502824,0.543129,9,3. The information specified in paragraph 7 of...
4,4,What is the name of the advisory council estab...,9. An advisory council shall be established on...,9. При уполномоченном органе по защите прав су...,0.814575,0.883145,1,9. An advisory council shall be established on...
5,5,What is the purpose of the advisory council?,9. An advisory council shall be established on...,9. При уполномоченном органе по защите прав су...,0.730695,0.624303,1,9. An advisory council shall be established on...
6,6,What is the first name of the operator?,"1) the name or surname, first name, patronymic...","1) наименование либо фамилия, имя, отчество и ...",0.616502,0.608745,2,"1) the name or surname, first name, patronymic..."
7,7,What is the name of the person who represents ...,"1) the name or surname, first name, patronymic...","1) наименование либо фамилия, имя, отчество и ...",0.291479,0.473255,1,"1) the name or surname, first name, patronymic..."
...,...,...,...,...,...,...,...,...
767,767,What is the legal basis for the execution of i...,5. Decisions of interstate bodies adopted on t...,"7. Проекты нормативных правовых актов, указанн...",0.484486,0.334377,2,5. Decisions of interstate bodies adopted on t...
769,769,What is the obligation to eliminate?,Article 21. Operator's obligations to eliminat...,"5. Решения межгосударственных органов, приняты...",0.395597,0.331271,4,Article 21. Operator's obligations to eliminat...
770,770,What is the obligation of the operator to elim...,Article 21. Operator's obligations to eliminat...,"5. Решения межгосударственных органов, приняты...",0.649768,0.353992,6,Article 21. Operator's obligations to eliminat...
774,774,What is the procedure for processing personal ...,4) processing of personal data classified as a...,Статья 10. Специальные категории персональных ...,0.853355,0.391578,2,4) processing of personal data classified as a...


In [6]:
from docQA.pipelines import Pipeline, TranslatorPipeline, RetrieverPipeline, RankerPipeline, CatboostPipeline
import pandas as pd


pipe = Pipeline(['docs/Федеральный-закон-от-27.07.2006-N-152-ФЗ-О-персональных-данных.txt'])
pipe.add_node(TranslatorPipeline, name='translator', is_technical=True, model_name='facebook/wmt19-ru-en',translate_text=True)
pipe.add_node(RetrieverPipeline, name='retriever')
pipe.add_node(RankerPipeline, name='ranker')
pipe.add_node(CatboostPipeline, name='catboost')


train_df = [{'question': newDf.iloc[i]['en_question'], 'context': newDf.iloc[i]['en_context']} for i in range(len(newDf))]
pipe.fit(train_df)

Opening docs:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3668: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the

Fine tuning retriever:   0%|          | 0/15 [00:00<?, ?it/s]

Fine tuning ranker:   0%|          | 0/15 [00:00<?, ?it/s]

3. The information specified in paragraph 7 of this article shall be provided to the subject of personal data or to his representative by the operator upon application or upon receipt of a request from the subject of personal data or his representative. The request shall contain the number of the main document certifying the identity of the subject of personal data or his representative, information on the date of issue of the said document and the issuing authority, information confirming the participation of the subject of personal data in the relationship with the operator (contract number, date of conclusion of the contract, conditional verbal designation and / or other information), or information otherwise confirming the fact of processing of personal data by the operator, the signature of the subject of personal data or his representative. The request may be sent in the form of an electronic document and signed by an electronic signature in accordance with the legislation of the

In [12]:
pipe('Кто такой субобработчик?')

[{'input': 'Кто такой субобработчик?',
  'output': {'answers': []},
  'modified_input': 'Who is the sub-handler?'}]

In [13]:
import shutil
shutil.make_archive('/content/docQA_14_10', 'zip', '/content/docQA_14_10')

'/content/docQA_14_10.zip'

In [2]:
from transformers import (
    AdamW,
    AutoModel,
    AutoTokenizer,
)

tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2').to('cuda')

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

In [81]:
import numpy as np
import torch
from tqdm import trange
from torch import device, Tensor
import re
import math

In [5]:
def batch_to_device(batch, target_device: device):
    for key in batch:
        if isinstance(batch[key], Tensor):
            batch[key] = batch[key].to(target_device)
    return batch

In [10]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

In [11]:
def forward(**kwargs):
    model_output = model(**kwargs)
    embeddings = mean_pooling(model_output, kwargs['attention_mask'])
    if True:
        embeddings = torch.nn.functional.normalize(embeddings, p=2, dim=1)

    return embeddings.cpu()

In [12]:
def encode(sentences):
    all_embeddings = []

    for start_index in trange(0, len(sentences), 16, desc="Batches", disable=True):
        sentences_batch = sentences[start_index:start_index + 16]
        features = tokenizer(sentences_batch, padding=True, truncation=True, return_tensors='pt')
        features = batch_to_device(features, 'cuda')

        with torch.no_grad():
            out_features = forward(**features)
            all_embeddings.extend(out_features)

    return [all_embeddings[idx] for idx in np.argsort(len(sentences))]


In [14]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [156]:
def extract_keywords(text, topN, ranges):
    similarites = []
    for rang in range(ranges[0]+1, ranges[1]+1):
        words = []
        splitted = text.split()
        for i in range(len(splitted)-rang):
            words.append(' '.join(splitted[i:i+rang]))
        
        text_emb = encode([text])[0]
        for word in words:
            resubed = re.sub(r'[^\w\s]','',word)
            if resubed:
                if resubed[0] == ' ': resubed = resubed[1:]
                if resubed[-1] == ' ': resubed = resubed[:-1]
            similarites.append({'cos_sim': cosine_similarity(text_emb, encode([resubed])[0]), 'keyword': resubed})
        
        similarites = sorted(similarites, key=lambda sim: sim['cos_sim'], reverse = True)

    result = []
    for i in similarites:
        if i not in result:
          result.append(i['keyword'])

    return result[:topN]

In [157]:
arr=extract_keywords('personal data processing - any action (operation) or a combination of actions (operations) performed both automatically and manually with personal data, including collection, recording, arrangement, accumulation, storage, specification (updating, changing), extraction, use, distribution (including transfer), anonymizing, blocking and destruction of personal data;',1000,(1,3))

In [158]:
arr

['personal data processing',
 'personal data',
 'personal data',
 'personal data including',
 'data processing',
 'data processing',
 'with personal data',
 'processing',
 'manually with personal',
 'processing  any',
 'data including',
 'actions operations performed',
 'actions operations',
 'destruction of personal',
 'operations performed',
 'anonymizing blocking and',
 'of actions operations',
 'action operation or',
 'any action operation',
 'including transfer anonymizing',
 'action operation',
 'transfer anonymizing',
 'anonymizing blocking',
 'automatically and manually',
 'operations performed both',
 'with personal',
 'performed both automatically',
 'automatically and',
 'of personal',
 'collection recording',
 'transfer anonymizing blocking',
 'recording arrangement accumulation',
 'accumulation storage',
 'data including collection',
 'storage specification',
 'arrangement accumulation storage',
 'extraction use distribution',
 'both automatically',
 'extraction use',
 'op